In [25]:
import numpy as np
import cv2 
import copy
import random

#添加椒盐噪声验证中值滤波效果
def add_noise(image, prob):
    '''
    imgae  输入图片
    prob   加入噪声概率判断
    '''
    #添加噪声前先判断是彩色图还是灰度图
    if len(image.shape) == 2:#灰度
        output = np.zeros((image.shape[0], image.shape[1]), dtype = 'uint8')
        thres = 1-prob
        for row in range(image.shape[0]):
            for col in range(image.shape[1]):
                rdn = random.random()
                if rdn < prob:
                    output[row][col] = 0
                elif rdn > thres :
                    output[row][col] = 255
                else :
                    output[row][col] = image[row][col]
    elif len(image.shape) == 3:#彩图
        output = copy.deepcopy(img)
        thres = 1-prob
        for row in range(image.shape[0]):
            for col in range(image.shape[1]):
                rdn = random.random()
                
                if rdn < prob:
                    output[row, col, 0] = 0
                    output[row, col, 1] = 0
                    output[row, col, 2] = 0
                elif rdn > thres :
                    output[row, col, 0] = 255
                    output[row, col, 1] = 255
                    output[row, col, 2] = 255
    return output

def medianBlur(img, kernelsize, padding_way):
    '''
    img 输入图像
    kernelsize   中值滤波方阵大小
    padding_way  padding方式，可选'ZERO'0填充/'REPLICA'边沿填充
    '''
    ################################padding过程#################################
    if padding_way == "ZERO":
        tool = 'constant'
    elif padding_way == "REPLICA":
        tool = 'edge'
    else:
        print("padding way is error...")
        return 0
    #计算padding像素大小   (h or w - 1 )/2  eg 3 -> 1   5 -> 2
    padding_num = int((kernelsize - 1) / 2)
    
    
    ################################Median fliter过程#############################
    if len(img.shape) == 3:
        src = copy.deepcopy(img)
        #若是彩色图片分出三个通道分别填充
        B,G,R = cv2.split(src)
        B = np.pad(B, ((padding_num,padding_num), (padding_num,padding_num )), tool)
        G = np.pad(G, ((padding_num,padding_num), (padding_num,padding_num )), tool)
        R = np.pad(R, ((padding_num,padding_num), (padding_num,padding_num )), tool)
        img_paded = cv2.merge((B,G,R))
        
        dst = copy.deepcopy(img)    #deepcopy原始图像，将滤波后的像素存入原始图像
        height, weight, channel = img_paded.shape
        for row in range(padding_num, height-padding_num):
            for col in range(padding_num, weight-padding_num):
                gap = int(kernelsize/2)
                dst[row-padding_num, col-padding_num,0] = np.median(B[row-gap:row+gap, col-gap:col+gap])        #矩阵切片
                dst[row-padding_num, col-padding_num,1] = np.median(G[row-gap:row+gap, col-gap:col+gap])
                dst[row-padding_num, col-padding_num,2] = np.median(R[row-gap:row+gap, col-gap:col+gap])
    else:
        src = copy.deepcopy(img)
        img_paded = np.pad(src, ((padding_num,padding_num), (padding_num,padding_num )), tool)
        dst = copy.deepcopy(img)
        height, weight= img_paded.shape
        for row in range(padding_num, height-padding_num):
            for col in range(padding_num, weight-padding_num):
                gap = int(kernelsize/2)
                dst[row-padding_num, col-padding_num] = np.median(img_paded[row-gap:row+gap, col-gap:col+gap])
    return dst
        

In [26]:
#加入椒盐噪声
img=cv2.imread('jiaoshi.jpg',0)
img_noise = add_noise(img, 0.001)
cv2.imshow('input image', img)
cv2.imshow('noise image',img_noise)
#中值滤波
res = medianBlur(img_noise, 5, "REPLICA")
cv2.imshow("median iresult", res)

key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()